In [ ]:
# Celda 1 Corregida
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Cargamos el dataset 'Credit Card Fraud'
print("Cargando dataset 'Credit Card Fraud'...")
cc_fraud = fetch_openml(data_id=1597, as_frame=True, parser='auto')
X = cc_fraud.data
y = pd.Series(pd.factorize(cc_fraud.target)[0], name='Class')
print("Dataset cargado.")

# Para este dataset, la mayoría de las columnas ya están escaladas.
# Escalamos únicamente la columna 'Amount', que es la que nos interesa para los costes.
scaler = StandardScaler()
X['scaled_amount'] = scaler.fit_transform(X['Amount'].values.reshape(-1, 1))

# Eliminamos la columna original 'Amount'. La columna 'Time' ya no se procesa.
X_processed = X.drop(['Amount'], axis=1)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.3, random_state=42, stratify=y
)

# Guardamos el importe real para usarlo en los pesos
# ¡IMPORTANTE! Usamos el DataFrame original 'X' ANTES de eliminar 'Amount'
amount_train = X.loc[y_train.index, 'Amount']

Cargando dataset 'Credit Card Fraud'...
Dataset cargado.


In [6]:
print("--- Entrenando Modelo Convencional ---")
lr_conv = LogisticRegression(random_state=42, max_iter=1000)
lr_conv.fit(X_train, y_train)
y_pred_conv = lr_conv.predict(X_test)

print("\nResultados del Modelo Convencional:")
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_conv))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_conv, target_names=['Normal', 'Fraude']))

--- Entrenando Modelo Convencional ---

Resultados del Modelo Convencional:
Matriz de Confusión:
[[85280    15]
 [   57    91]]

Reporte de Clasificación:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     85295
      Fraude       0.86      0.61      0.72       148

    accuracy                           1.00     85443
   macro avg       0.93      0.81      0.86     85443
weighted avg       1.00      1.00      1.00     85443



In [7]:
print("\n--- Entrenando Modelo Avanzado Sensible al Importe ---")

# Crear los pesos: si es fraude, el peso es su importe. Si no, el peso es 1.
# Añadimos +1 para evitar importes de 0.
sample_weights = y_train.apply(lambda is_fraud: amount_train[is_fraud] + 1 if is_fraud else 1)

# Entrenar el modelo pasándole los pesos de cada muestra
lr_cost_sensitive = LogisticRegression(random_state=42, max_iter=1000)
lr_cost_sensitive.fit(X_train, y_train, sample_weight=sample_weights)
y_pred_cs = lr_cost_sensitive.predict(X_test)

print("\nResultados del Modelo Sensible al Importe:")
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred_cs))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_cs, target_names=['Normal', 'Fraude']))


--- Entrenando Modelo Avanzado Sensible al Importe ---

Resultados del Modelo Sensible al Importe:
Matriz de Confusión:
[[85265    30]
 [   37   111]]

Reporte de Clasificación:
              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00     85295
      Fraude       0.79      0.75      0.77       148

    accuracy                           1.00     85443
   macro avg       0.89      0.87      0.88     85443
weighted avg       1.00      1.00      1.00     85443

